> Dana jest tablica $ A $ zawierająca $ n $ parami różnych liczb. Proszę zaproponować algorytm, który znajduje takie dwie liczby $ x $ i $ y $ z $ A $, że $ y − x $ jest jak największa oraz w tablicy nie ma żadnej liczby $ z $ takiej, że $ x < z < y $ (innymi słowy, po posortowaniu tablicy $ A $ rosnąco wynikiem byłyby liczby $ A[i] $ oraz $ A[i+1] $ dla których $ A[i + 1] − A[i] $ jest największe).

### Omówienie algorytmu

##### Wstępne rozważania

Oczywiście możliwe jest przesortowanie tablicy, a następnie liniowe przejście i sprawdzanie różnic sąsiednich liczb, ale warto zauważyć, że nie wiemy nic na temat zakresu liczb oraz ich rozkładu. Także nie jest powiedziane, że są to liczby całkowite, więc od razu odrzucamy Radix Sorta i Counting Sorta, które z reguły działają tylko dla zbiorów danych, składających się wyłącznie z liczb całkowitych (lub takich, które można przedstawić jako liczby całkowite - np. ciągi tekstowe). Konwersja liczby zmiennoprzecinkowej na 2 inty, reprezentujące część całkowitą i ułamkową jest w tym przypadku trochę overkill, ale trzeba mieć na uwadze, że w tym przypadku pojawiłby się również błąd, związany z przechowywaniem floatów w pamięci komputera, więc prawdopodobnie byśmy mieli dodatkowe problemy.

Po wstępnych rozważaniach widzimy, że jedyny rozsądny algorytm sortowania to Bucket Sort, w którym użylibyśmy rekurencyjnych (lub iteracyjnych) wywołań Bucket Sorta dla dużych wiaderek i Insertion Sorta dla małych. Mimo to, zadanie da się rozwiązać nieco lepiej pod względem złożoności obliczeniowej oraz łatwości wyjaśnienia, że rzeczywista złożoność to $ O(n) $ (w przypadku Bucket Sorta, który nawet korzysta z Bucket Sorta dla dużych wiaderek, prawdopodobne trudno byłoby wybronić rozwiązanie i udowodnić jego poprawność).

Warto zwrócić uwagę, że nie mamy nic powiedzianego o złożoności pamięciowej, więc przyjmujemy, że pamięć nie ma znaczenia. Z tego powodu, możemy zająć nawet $ O(n) $ (lub więcej, jeśli to konieczne) pamięci, ale $ O(n) $ nam wystarczy.

##### Omówienie implementacji

###### 1. krok

Tworzymy $ n $ kubełków tak, aby średnio jedna wartość znalazła się w jednym kubełku (uprości to dalsze postępowanie, jeżeli będzie mało wartości w kubełkach). Skoro mamy $ n $ kubełków, musimy wyznaczyć równomiernie przedział wartości, które mają trafić do danego kubełka. Najłatwiej postąpić podobnie do Bucket Sorta i obliczyć krok, który oznacza rozmiar zakresu wartości (różnicę między górnym a dolnym ograniczeniem przedziału), jako:

    interval = (max_val - min_val) / n  # Dzielimy przez n, bo to liczba wiaderek

Po stworzeniu kubełków, postępujemy identycznie do Bucket Sorta, a więc przechodzimy liniowo przez tablicę wartości i umieszczamy je kolejno w odpowiednich wiaderkach (kubełkach).

###### 2. krok

Szukamy takich dwóch kubełków, pomiędzy którymi jak największa liczba kubełków jest pusta. Możemy od razu zauważyć, że nie zawsze zdarzy się taka sytuacja i istnieje możliwość, w której wszystkie wartości trafią każda do innego kubełka, więc nie będzie pustych kubełków. Nie jest to jednak problem, o czym powiemy sobie później. Warto również zauważyć, że nigdy nie zdarzy się sytuacja by wszystkie wartości trafiły do jednego kubełka i jednocześnie ten kubełek był jedynym utworzonym kubełkiem, ponieważ zawsze w tablicy muszą być przynajmniej 2 wartości, abyśmy mogli wyznaczyć ich różnicę, a więc liczba kubełków będzie równa przynajmniej 2, bo mamy zależność: $ k = n \ge 2 $, gdzie $ k $ - liczba kubełków.

Potrzebujemy 2 wskaźników, z których pierwszy przechodzi po tablicy (po kolejnych wiaderkach), a drugi zatrzymuje się na ostatnim niepustym wiaderku różnym od tego, na którym znajduje się pierwszy wskaźnik. Jeżeli pierwszy wskaźnik znajdzie niepuste wiaderko, wówczas wyznaczamy najmniejszą wartość z tego wiaderka oraz wartość największą z poprzedniego niepustego wiaderka i obliczmy ich różnicę. Jeżeli jest ona większa od poprzedniej największej różnicy, zapisujemy wartości $ x $, $ y $ oraz nową największą różnicę. Jeżeli nie, przesuwamy pierwszy wskaźnik na następne wiaderko, a drugi wskaźnik na to wiaderko, na którym poprzednio był pierwszy wskaźnik.

###### UWAGA:

Zauważmy, że nie jest konieczne rozważanie sytuacji, w której porównujemy wartości dwóch liczb z tego samego wiaderka, ponieważ ZAWSZE najmniejsza wartość trafi do pierwszego wiaderka, a największa do ostatniego utworzonego wiaderka, więc nigdy nie będzie takiej sytuacji, że jeżeli są 2 wartości na granicach przedziału z danego wiaderka, to ich różnica jest największa. Jest to niemożliwe, bo mamy $ n $ wiaderek, więc w takim przypadku przynajmniej jedno wiaderko (jak 2 pewne wartości będą w tym samym wiaderku) musi być puste, a to puste wiaderko NIGDY nie będzie pierwszym ani ostatnim wiaderkiem (to wyjaśniliśmy wyżej). Zatem w takiej sytuacji największą różnicą będzie różnica między najmniejszym elementem z wiaderka na prawo względem pustego wiaderka a największym z wiaderka po lewej stronie pustego wiaderka.

###### 3. krok

Po przejściu wyżej opisaną pętlą po wszystkich wiaderkach, wystarczy zwrócić wyznaczone wartości $ x $ oraz $ y $. 

### Implementacja algorytmu

In [1]:
def greatest_difference_neighbors(arr):
    # Return None if an array has no more than 1 element
    if len(arr) < 2: return None, None
    
    # Create buckets and calculate the interval
    buckets_count = len(arr)
    buckets = [[] for _ in range(buckets_count)]
    min_val, max_val = minmax(arr)
    val_interval = (max_val - min_val) / buckets_count
    # Deploy all the values in the correct buckets
    for val in arr:
        bucket_idx = int((val - min_val) / val_interval - .5)
        buckets[bucket_idx].append(val)
        
    # Look for the greatest neighbours difference
    i = 0  # Bucket at index 0 will be never empty
    max_diff = 0
    x = y = None
    for j in range(1, buckets_count):
        # If a bucket is not empty, calculate a difference
        if buckets[j]:
            max_val_i = max(buckets[i])
            min_val_j = min(buckets[j])
            diff = min_val_j - max_val_i
            # Update the max_diff variable if the current difference
            # is greater than the previous one
            if diff > max_diff:
                max_diff = diff
                x = max_val_i
                y = min_val_j
            # Update the i pointer (j will be advanced automatically in
            # a for loop)
            i = j
            
    # Return the final results
    return x, y
            
    
def minmax(iterable):
    # We assume that an iterable supports indexing (to handle sets or dictionaries
    # iteration we can use an iterator object but in this example this is overkill).
    # Store the last value in order to ease odd-length iterable values comparisons.
    max_val = min_val = iterable[-1]  
    
    for i in range(1, len(iterable), 2):
        if iterable[i] > iterable[i-1]:
            if iterable[i] > max_val:   max_val = iterable[i]
            if iterable[i-1] < min_val: min_val = iterable[i-1]
        else:
            if iterable[i] < min_val:   min_val = iterable[i]
            if iterable[i-1] > max_val: max_val = iterable[i-1]
                
    # Return the minimum and the maximum value
    return min_val, max_val

###### Kilka testów

##### Funkcja generująca losowe dane wejściowe

In [2]:
import random

def random_test_data(range_=(-10, 10), vals_count=(0, 20), int_only=False):
    if int_only:
        arr = [random.randint(*map(int, range_)) for _ in range(random.randint(*vals_count))]
    else:
        arr = [random.random() * random.randint(*map(int, range_)) for _ in range(random.randint(*vals_count))]
    sorted_arr = sorted(arr)
    expected_diff = 0
    x = y = None
    for i in range(1, len(sorted_arr)):
        diff = sorted_arr[i] - sorted_arr[i-1]
        if diff > expected_diff:
            x = sorted_arr[i-1]
            y = sorted_arr[i]
            expected_diff = diff
    return arr, sorted_arr, expected_diff, x, y

##### Kod sprawdzający poprawność algorytmu

In [3]:
arr, sorted_arr, expected_diff, expected_x, expected_y = random_test_data(
    range_=(-100, 100), 
    vals_count=(0, 100),
    int_only=False
)

x, y = greatest_difference_neighbors(arr)
diff = (y - x) if x and y else None

print(f"Result:   x = {x}, y = {y}")
print(f"Expected: x = {expected_x}, y = {expected_y}")
print(f"Result difference:   {diff}")
print(f"Expected difference: {expected_diff}")
print(f"An algorithm is {'CORRECT' if x == expected_x and y == expected_y else 'WRONG'}")
print("\nInput array:", arr, sep='\n')
print("\nSorted array:", sorted_arr, sep='\n')

Result:   x = -4.761466021489896, y = 17.076449072191423
Expected: x = -4.761466021489896, y = 17.076449072191423
Result difference:   21.837915093681318
Expected difference: 21.837915093681318
An algorithm is CORRECT

Input array:
[17.076449072191423, -11.249705368186314, -34.85072202867098, 19.5644339182881, -19.430198445675163, 21.438538669436152, -4.761466021489896, 17.17855164119556, -14.809660698537623]

Sorted array:
[-34.85072202867098, -19.430198445675163, -14.809660698537623, -11.249705368186314, -4.761466021489896, 17.076449072191423, 17.17855164119556, 19.5644339182881, 21.438538669436152]
